# 中医证型关联规则挖掘

## 挖掘目标

- 借助三阴乳腺癌患者的临床患病信息，挖掘患者的症状与中医证型之间的关联关系
- 对截断治疗提供依据，挖掘潜在证素

## 挖掘方法与过程

目前，患者的临床病例信息大豆记录在纸上，很少会将患者的患病信息存放在系统中，因此进行数据分析是会面临数据缺乏的情况，针对这种情况，采用问卷调查的方式收集数据。

采用关联规则算法，挖掘各个中医证素与乳腺癌TNM分期之间的联系。

步骤：
- 以问卷调查的方式对数据进行收集，并将问卷信息整理成原始数据
- 对原始数据集进行数据预处理，包括数据清洗、属性规约、数据变换
- 采用关联规则算法，调整模型输入参数，获取各中医证素与乳腺癌TNM分期之间的关系
- 结合实际业务，对模型结果进行分析，将模型结果应用到实际业务中

### 数据获取
- 拟定调查问卷表并形成原始指标表
- 定义纳入标准与排除标准
- 将收集回来的问卷表整理成原始数据

### 数据预处理

**数据清洗**
- 问卷有效性筛选

**属性规约**
- 选择所有属性中的6种证型得分和TNM分期的属性值构成数据集

**数据变换**
- 将证型得分变为相对数
- 数值离散化，因为Apriori算法不能处理连续属性

In [1]:
import pandas as pd
from sklearn.cluster import KMeans

datafile = './data/data_8/data.xls'
processedfile = './data/data_7/data_precessed.xls'
typelabel = {'肝气郁结证型系数': 'A', '热毒蕴结证型系数': 'B', '冲任失调证型系数': 'C', '气血两虚证型系数': 'D', '脾胃虚弱证型系数': 'E', '肝肾阴虚证型系数': 'F'}
k = 4

data = pd.read_excel(datafile)
keys = list(typelabel.keys())
result = pd.DataFrame()

for i in range(len(keys)):
    kmodel = KMeans(n_clusters=k, n_jobs=4).fit(data[[keys[i]]].values)
    r = kmodel.predict(data[[keys[i]]].values)
    s = [typelabel[keys[i]] + str(j) for j in r]
    data[keys[i]] = s

In [2]:
data = data.drop(['转移部位', '确诊后几年发现转移'], axis=1)
data.head()

,肝气郁结证型系数,热毒蕴结证型系数,冲任失调证型系数,气血两虚证型系数,脾胃虚弱证型系数,肝肾阴虚证型系数,病程阶段,TNM分期
0,A2,B1,C0,D0,E3,F0,S4,H4
1,A3,B2,C0,D0,E3,F0,S4,H4
2,A2,B2,C0,D2,E3,F1,S4,H4
3,A0,B0,C3,D2,E1,F0,S4,H4
4,A1,B0,C1,D1,E1,F0,S4,H4


In [3]:
data.to_excel(processedfile)

### 模型构建

In [4]:
from apriori import *

inputfile = './data/data_8/apriori.txt'
data = pd.read_csv(inputfile, header=None, dtype=object)

ct = lambda x: pd.Series(1, index=x[pd.notnull(x)])
b = map(ct, data.as_matrix())
data = pd.DataFrame(b).fillna(0)
del b

support = 0.06
confidence = 0.75
ms = '---'

find_rule(data, support, confidence, ms)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys



正在进行第1次搜索...
数目：276...

正在进行第2次搜索...
数目：947...

正在进行第3次搜索...
数目：41...

结果为：
                    support  confidence
A3---F4---H4       0.078495    0.879518
C3---F4---H4       0.075269    0.875000
B2---F4---H4       0.062366    0.794521
C2---E3---D2       0.092473    0.754386
D2---F3---H4---A2  0.062366    0.753247


,support,confidence
A3---F4---H4,0.078495,0.879518
C3---F4---H4,0.075269,0.875000
B2---F4---H4,0.062366,0.794521
C2---E3---D2,0.092473,0.754386
D2---F3---H4---A2,0.062366,0.753247


**结论**

TNM分期为H4期的三阴乳腺癌患者证型主要为肝肾阴虚证、热毒蕴结证、肝气郁结证和冲任失调，H4期患者肝肾阴虚证和肝气郁结证的临床表现较为突出，最大置信度可达87.9%